<a href="https://colab.research.google.com/github/leonfrench/polyprotein_stats/blob/yeast_embeddings/notebooks/generate_proportions_yeast.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
!wget https://github.com/leonfrench/AD-Allen-FDG/raw/master/data/protein_gencode/gencode.v32.pc_translations.fa.gz
!gzip -d gencode.v32.pc_translations.fa.gz

--2022-02-07 15:53:34--  https://github.com/leonfrench/AD-Allen-FDG/raw/master/data/protein_gencode/gencode.v32.pc_translations.fa.gz
Resolving github.com (github.com)... 192.30.255.113
Connecting to github.com (github.com)|192.30.255.113|:443... connected.
HTTP request sent, awaiting response... 302 Found
Location: https://raw.githubusercontent.com/leonfrench/AD-Allen-FDG/master/data/protein_gencode/gencode.v32.pc_translations.fa.gz [following]
--2022-02-07 15:53:34--  https://raw.githubusercontent.com/leonfrench/AD-Allen-FDG/master/data/protein_gencode/gencode.v32.pc_translations.fa.gz
Resolving raw.githubusercontent.com (raw.githubusercontent.com)... 185.199.108.133, 185.199.109.133, 185.199.110.133, ...
Connecting to raw.githubusercontent.com (raw.githubusercontent.com)|185.199.108.133|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 11643034 (11M) [application/octet-stream]
Saving to: ‘gencode.v32.pc_translations.fa.gz’

gencode.v32.pc_tran 100%[==========

In [ ]:
!pip install biopython

     |████████████████████████████████| 2.3 MB 7.6 MB/s 


In [ ]:
import pandas as pd
import numpy as np
import os
from Bio import SeqIO
from datetime import datetime
import functools
from google.colab import drive
from collections import Counter

In [ ]:
embedding_file_path_processed = './gdrive/MyDrive/data/protein_embeddings_processed/'

In [ ]:
drive.mount("/content/gdrive/")

Mounted at /content/gdrive/


In [ ]:
sequence_list = list(SeqIO.parse("gencode.v32.pc_translations.fa", "fasta"))

In [ ]:
sequences_strings = [str(x.seq) for x in sequence_list]
sequences_ids = [x.id for x in sequence_list]

In [ ]:
sequence_df = pd.DataFrame(  data = {'ids' : sequences_ids, 'sequences': sequences_strings})

In [ ]:
#count amino acids
sequence_df = pd.concat([sequence_df.reset_index(drop=True), sequence_df['sequences'].apply(Counter).apply(pd.Series)], axis = 1)

In [ ]:
#drop the sequence column
#extract gene_symbol again (code reuse from another workbook)
sequence_df['gene_symbol'] = sequence_df['ids'].str.split(pat="|", n=-1, expand=True)[6]
sequence_df.drop(['ids', 'sequences'], axis=1, inplace=True) 

cols = sequence_df.columns.tolist()
cols = cols[-1:] + cols[:-1]
sequence_df = sequence_df[cols]
#write out

In [ ]:
sequence_df['length'] = sequence_df.sum(axis=1, numeric_only=True)

In [ ]:
sequence_df

,gene_symbol,M,K,V,T,A,E,I,S,W,N,F,L,G,D,Q,Y,H,P,R,C,length
0,OR4F5,12.0,15.0,27.0,22.0,18.0,7.0,31.0,29.0,4.0,10.0,28.0,41.0,15.0,13.0,7.0,10.0,9.0,11.0,8.0,9.0,326.0
1,OR4F5,11.0,13.0,26.0,19.0,16.0,4.0,30.0,25.0,3.0,7.0,28.0,41.0,15.0,13.0,7.0,10.0,9.0,11.0,8.0,9.0,305.0
2,OR4F29,11.0,12.0,31.0,17.0,18.0,5.0,20.0,29.0,3.0,8.0,29.0,44.0,15.0,10.0,6.0,11.0,10.0,13.0,11.0,9.0,312.0
3,OR4F16,11.0,12.0,31.0,17.0,18.0,5.0,20.0,29.0,3.0,8.0,29.0,44.0,15.0,10.0,6.0,11.0,10.0,13.0,11.0,9.0,312.0
4,SAMD11,4.0,17.0,11.0,11.0,39.0,20.0,12.0,31.0,3.0,7.0,10.0,44.0,25.0,15.0,9.0,4.0,13.0,39.0,34.0,10.0,358.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
100286,MT-ND4L,10.0,0.0,6.0,5.0,9.0,2.0,7.0,8.0,0.0,6.0,3.0,23.0,4.0,1.0,1.0,4.0,3.0,2.0,1.0,3.0,98.0
100287,MT-ND4,27.0,11.0,13.0,48.0,26.0,9.0,39.0,43.0,13.0,23.0,20.0,96.0,17.0,3.0,10.0,13.0,13.0,23.0,9.0,3.0,459.0
100288,MT-ND5,26.0,21.0,15.0,65.0,44.0,9.0,54.0,49.0,12.0,33.0,38.0,104.0,26.0,11.0,20.0,16.0,14.0,32.0,8.0,6.0,603.0
100289,MT-ND6,10.0,2.0,31.0,4.0,8.0,10.0,12.0,10.0,5.0,4.0,8.0,19.0,29.0,3.0,0.0,11.0,0.0,4.0,3.0,1.0,174.0


In [ ]:
#drop x and u
sequence_df.drop(['X', 'U'], axis=1, inplace=True) 
#fill na
sequence_df.fillna(0, inplace=True)

In [ ]:
#change to proportion

In [ ]:
cols = set(sequence_df.columns.values).difference(('gene_symbol','length'))

In [ ]:
sequence_df[[col for col in cols]] = sequence_df[cols].div(sequence_df['length'], axis='index')

In [ ]:
sequence_df

,gene_symbol,M,K,V,T,A,E,I,S,W,N,F,L,G,D,Q,Y,H,P,R,C,length
0,OR4F5,0.036810,0.046012,0.082822,0.067485,0.055215,0.021472,0.095092,0.088957,0.012270,0.030675,0.085890,0.125767,0.046012,0.039877,0.021472,0.030675,0.027607,0.033742,0.024540,0.027607,326.0
1,OR4F5,0.036066,0.042623,0.085246,0.062295,0.052459,0.013115,0.098361,0.081967,0.009836,0.022951,0.091803,0.134426,0.049180,0.042623,0.022951,0.032787,0.029508,0.036066,0.026230,0.029508,305.0
2,OR4F29,0.035256,0.038462,0.099359,0.054487,0.057692,0.016026,0.064103,0.092949,0.009615,0.025641,0.092949,0.141026,0.048077,0.032051,0.019231,0.035256,0.032051,0.041667,0.035256,0.028846,312.0
3,OR4F16,0.035256,0.038462,0.099359,0.054487,0.057692,0.016026,0.064103,0.092949,0.009615,0.025641,0.092949,0.141026,0.048077,0.032051,0.019231,0.035256,0.032051,0.041667,0.035256,0.028846,312.0
4,SAMD11,0.011173,0.047486,0.030726,0.030726,0.108939,0.055866,0.033520,0.086592,0.008380,0.019553,0.027933,0.122905,0.069832,0.041899,0.025140,0.011173,0.036313,0.108939,0.094972,0.027933,358.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
100286,MT-ND4L,0.102041,0.000000,0.061224,0.051020,0.091837,0.020408,0.071429,0.081633,0.000000,0.061224,0.030612,0.234694,0.040816,0.010204,0.010204,0.040816,0.030612,0.020408,0.010204,0.030612,98.0
100287,MT-ND4,0.058824,0.023965,0.028322,0.104575,0.056645,0.019608,0.084967,0.093682,0.028322,0.050109,0.043573,0.209150,0.037037,0.006536,0.021786,0.028322,0.028322,0.050109,0.019608,0.006536,459.0
100288,MT-ND5,0.043118,0.034826,0.024876,0.107794,0.072968,0.014925,0.089552,0.081260,0.019900,0.054726,0.063018,0.172471,0.043118,0.018242,0.033167,0.026534,0.023217,0.053068,0.013267,0.009950,603.0
100289,MT-ND6,0.057471,0.011494,0.178161,0.022989,0.045977,0.057471,0.068966,0.057471,0.028736,0.022989,0.045977,0.109195,0.166667,0.017241,0.000000,0.063218,0.000000,0.022989,0.017241,0.005747,174.0


In [ ]:
#combine multiple genes into single rows
sequence_df = sequence_df.groupby('gene_symbol').agg('mean')
sequence_df = sequence_df.reset_index()

In [ ]:
sequence_df.shape

(20341, 22)

In [ ]:
sequence_df

,gene_symbol,M,K,V,T,A,E,I,S,W,N,F,L,G,D,Q,Y,H,P,R,C,length
0,A1BG,0.010598,0.027505,0.069531,0.064100,0.071552,0.078109,0.017288,0.075079,0.018927,0.020947,0.037225,0.120633,0.086935,0.038864,0.034824,0.018298,0.035834,0.090975,0.061202,0.021576,400.000000
1,A1CF,0.027583,0.065919,0.065969,0.059079,0.099312,0.066966,0.051410,0.040462,0.006537,0.042524,0.033074,0.078614,0.094005,0.042798,0.036495,0.048509,0.012446,0.061762,0.055396,0.011139,542.875000
2,A2M,0.024255,0.068036,0.102056,0.076468,0.050801,0.055588,0.031255,0.088731,0.001866,0.037295,0.036762,0.146992,0.066296,0.033100,0.030937,0.031280,0.027919,0.044583,0.029584,0.016197,451.250000
3,A2ML1,0.024095,0.053125,0.071410,0.063234,0.057455,0.044727,0.043194,0.104884,0.011146,0.039031,0.042689,0.109897,0.064593,0.038897,0.059611,0.036657,0.016105,0.057817,0.035954,0.024563,644.500000
4,A3GALT2,0.029412,0.026471,0.047059,0.035294,0.111765,0.055882,0.026471,0.041176,0.044118,0.014706,0.055882,0.120588,0.085294,0.038235,0.032353,0.023529,0.041176,0.064706,0.091176,0.014706,340.000000
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
20336,ZXDC,0.012037,0.041627,0.048945,0.055608,0.089749,0.048309,0.018578,0.119002,0.002902,0.024402,0.035763,0.098214,0.089239,0.044428,0.039708,0.009973,0.040871,0.092263,0.055113,0.032472,662.000000
20337,ZYG11A,0.035241,0.054338,0.058132,0.059731,0.075753,0.056657,0.048382,0.076192,0.013545,0.041232,0.055497,0.143512,0.024489,0.036875,0.062928,0.015144,0.027248,0.036717,0.048224,0.030164,531.000000
20338,ZYG11B,0.027575,0.051367,0.058722,0.062467,0.076504,0.060949,0.051617,0.066076,0.008027,0.043590,0.044474,0.148092,0.039807,0.049812,0.058050,0.010465,0.036062,0.032279,0.048929,0.025136,655.000000
20339,ZYX,0.012413,0.041358,0.049779,0.045027,0.090415,0.058709,0.019168,0.078399,0.004120,0.017996,0.035402,0.059316,0.071583,0.032089,0.081834,0.014908,0.023587,0.189411,0.042359,0.031316,316.428571


In [ ]:
sequence_df = sequence_df.reset_index(drop=True)

In [ ]:
sequence_df.to_csv(os.path.join(embedding_file_path_processed, "gene_symbol_summarized_proportions.csv"), index=False)

In [ ]:
embedding_file_path_processed

'./gdrive/MyDrive/data/protein_embeddings_processed/'